In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install accelerate>=0.12.0
# !pip install datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [3]:
import torch
import json
from transformers import pipeline, AutoTokenizer
# from datasets import load_dataset
import random


dolly = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [6]:

def divide_chunks(l_, n_):
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def load_json(file__name):
    data_file = open(file__name, "r", encoding='utf-8')
    file_data = json.loads(data_file.read())
    data_file.close()
    return file_data


def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def main():
  filename="./test_3_mult_diversity_dolly.json"
  data = load_json(filename)

  for key in data:
    result = data[key]
    query_list=result.get("questions")
    gs = result.get("generated_sparql")

    res = dolly(query_list)
    print(res)
    gst = [x[0]["generated_text"] for x in res]

    for ii, l in enumerate(gst):
        for iii in range(10):
            if "SELECT" not in l:
                print(iii,ii)
                res = dolly(query_list[ii])
                gst[ii] = res[0]["generated_text"]
                l = gst[ii]
            else:
                break
    result["generated_sparql"] = gst
    data[key] = result

    save_json(filename, data)



In [7]:
main()

[[{'generated_text': 'SELECT DISTINCT?paper?paper_lbl WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "GAD")?benchmark orkgp:HAS_DATASET?dataset.?cont orkgp:HAS_BENCHMARK?benchmark.?paper orkgp:P31?cont; rdfs:label?paper_lbl. }'}], [{'generated_text': 'SELECT DISTINCT?paper?paper_lbl WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "Penn Treebank (Word Level") }'}], [{'generated_text': 'select distinct?paper?paper_lbl where {\n?dataset a orkgc:Dataset;\n  rdfs:label?dataset_lbl. \n  FILTER (str(?dataset_lbl) = "GAD")?benchmark orkgp:HAS_DATASET?dataset.\n ?cont orkgp:HAS_BENCHMARK?benchmark. \n ?paper orkgp:P31?cont;\n  rdfs:label?paper_lbl. \n  }'}], [{'generated_text': 'SELECT DISTINCT?paper?paper_lbl WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "Penn Treebank")?benchmark orkgp:HAS_DATASET?dataset.?cont orkgp:HAS_BENCHMARK?benchmark.?paper orkgp:P31?cont; rdfs:label?paper_

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


8 6
9 6
0 7
1 7
0 15
0 16
1 16
2 16
3 16
4 16
5 16
6 16
7 16
8 16
9 16
0 19
[[{'generated_text': 'RACE   P31'}], [{'generated_text': 'SELECT DISTINCT \n   ?paper?paper_lbl \nWHERE { \n   ?dataset a orkgc:Dataset; \n    rdfs:label?dataset_lbl. \n    FILTER (str(?dataset_lbl) = "Penn Treebank (Word Level)" ) \n   ?benchmark orkgp:HAS_DATASET?dataset. \n   ?cont orkgp:HAS_BENCHMARK?benchmark. \n   ?paper orkgp:P31?cont; \n    rdfs:label?paper_lbl. \n}'}], [{'generated_text': "In this query, we're looking for the title and ID of any research papers that contain a benchmark over the DDI dataset. \n\nWe'll find any research papers that contain the property HAS_DATASET on the class dataset which is the property of the dataset rdf:resource called DDI in RDF.\n\nWe'll look for the property P31 on the property benchmark which is a property of the class cont in the UrbanSound8k dataset. This is a benchmark used to measure the performance of a system.\n\nIf we find one or more results, the answers